In [ ]:
#
# Looks at envelopes of surface elevation and their spectra
# this checks on the normalisations necessary to get the 
# spectra of envelope squared and envelope from the 2D wave spectrum
# following Rice (1941 - p 131)
# First created :      M. de Carlo    2022/04/23
#

### MATPLOTLIB & SCIPY LIRAIRIES ##############""
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.text as mtext
import matplotlib.colors as mcolors
import matplotlib as mpl
import matplotlib.cm as cmx 
import numpy as np
import scipy.special as sps # function erf
import scipy.signal as spsi # function fftconvolve

import warnings
warnings.filterwarnings("ignore")
plt.rcParams.update({'font.size': 18,'savefig.facecolor':'white'})

from simulator_functions import *
from altimetry_processing_functions import *

In [ ]:
# In this first example we define the wave spectrum direcly on the kx,ky grid 
# that is the Fourier transform of the x,y grid we want for the sea surface

# kX, kY : grid of wavenumbers = 2 * pi /wavelength
# steps : dkx and dky, with nx and ny values. dkx = 2*pi/(dx*nx)
# x, y : grid of real x,y : nx values with a dx step. 
noise= 0
## ---  physical surface definition
# size of simulated sea surface in x and y
nx = 2048
ny = 2048
dx = 10 # [m]
dy = 10 # [m]

## --- defines spectrum
theta_m=60
D=2000
T0=16
Hs = 4
Lp=9.81/np.pi*T0**2;
kp=2*np.pi/Lp;sx=kp*0.1;sy=sx;


Z2D_def, kX, kY = def_spectrum_for_surface(nx=nx,ny=ny,dx=dx,dy=dy,theta_m=theta_m,D=D,T0=T0,Hs=Hs,
                         sk_theta=sx,sk_k=sy,typeSpec='Gaussian')

# Z1, kX, kY = def_spectrum_for_surface(nx=nx,ny=ny,dx=dx,dy=dy,theta_m=theta_m,D=D,T0=T0,
#                                       nk=1001,nth=36,klims=(0.0002,0.2),n=4,typeSpec='PM')

# Makes a surface with random phases using a seed (4th argument)
S2D_r,S2D_i,X,Y,kx,rg,dkx,dky= surface_from_Z1kxky_uniform_phase(Z2D_def,kX,kY,10) 

In [ ]:

fig,ax=plt.subplots(figsize=(8,8))
plt.imshow(np.fliplr(np.transpose(Z2D_def)),vmin=0, vmax=10000)


In [ ]:
## Figure Surface et Surface Enveloppe ###########################""
fig,axs=plt.subplots(1,2,figsize=(20,7))#,sharey=True,sharex=True)

im=axs[0].pcolormesh(X/1000,Y/1000,S2D_r,cmap='seismic',norm = mcolors.Normalize(vmin=S2D_r.min(), vmax=S2D_r.max()))
_=plt.colorbar(im,ax=axs[0],label='$\zeta$ [m]')
_=axs[0].set_xlabel('X [km]')
_=axs[0].set_ylabel('Y [km]')
_=axs[0].set_title('Surface')


A = np.sqrt(S2D_r**2+S2D_i**2)
B=np.sqrt(A)

im=axs[1].pcolormesh(X/1000,Y/1000,B,cmap='seismic',norm = mcolors.Normalize(vmin=S2D_r.min(), vmax=S2D_r.max()))
_=plt.colorbar(im,ax=axs[1],label='$\zeta$ [m]')

_=axs[1].set_xlabel('X [km]')
_=axs[1].set_ylabel('Y [km]')
_=axs[1].set_title('Surface enveloppe')
_=plt.tight_layout()

In [ ]:
# Checking on what convolution in numpy really is: not circular convolution !!!
# let us define "myconv" as the circular convolution we need. 
# This is OK here because our surface is periodic and the FFT gives no funny effects ... 
# warning this convolution does not flip the second array ... need to be flipped before

import numpy as np
def myconv(x, h):
    assert np.shape(x) == np.shape(h), 'Inputs to periodic convolution '\
                               'must be of the same period, i.e., shape.'

    X = np.fft.fft(x)
    H = np.fft.fft(h)

    return np.real(np.fft.ifft(np.multiply(X, H)))

# and here is a quick verification

a=[2.0,3.0]
b=[1.0,0.5]
print('this is the standard non-circular convolution:',np.convolve(a,b, 'same'))
c=myconv(a,b)
print('this is my circular convolution:              ',c)
print(a[0]*b[0]+a[1]*b[-1],a[0]*b[1]+a[1]*b[0])

In [ ]:
isplot = 1
isprints = 1
Lc=4000 # cut-off wavelength approximating effect of altimeter footprint
kc=2*np.pi/Lc

In [ ]:
    ### Get a equivalent 2D spectrum from DFT of the surface ################
    # -- to verify the coefficients ------------------------------------
    Z2D_from_FFT = np.abs(np.fft.fft2(S2D_r,norm='forward'))**2/(dkx*dky)

    ### 1D spectrum from integration over ky 
    Z1D_def = (np.sum(Z2D_def*dky,axis=0))
    Z1D_def_noshift=np.fft.ifftshift(Z1D_def)
    
    # -- reconstructed (DFT) 2D spectrum
    Z1D_from_FFT_noshift = np.sum(Z2D_from_FFT*dky,axis=0) 
    Z1D_from_FFT = np.fft.fftshift(np.sum(Z2D_from_FFT*dky,axis=0)) # fftshift to have the values from -k_lim to k_lim
    # set the spectrum only to positive part (equiv to Z2_def) => easier to generate surface
    Z1D_from_FFT[0:nx//2]=0
    Z1D_from_FFT[nx//2:]=2*Z1D_from_FFT[nx//2:] 

    ### Generate a 1D surface based on 1D spectrum (1 realisation) #########
    rng = np.random.default_rng(0)
    rg = rng.uniform(low=0.0, high=1.0, size=(nx))
    # -- from original 1D spectrum (from 2D)
    zhats=np.sqrt(2*np.fft.ifftshift(Z1D_def)*dkx)*np.exp(1j*2*np.pi*rg)
    S1D_def_r = np.real(np.fft.ifft(zhats,norm="forward"))
    # -- from 2D spectrum from 2D FFT spectrum
    zhats=np.sqrt(2*np.fft.ifftshift(Z1D_from_FFT)*dkx)*np.exp(1j*2*np.pi*rg)
    S1D_from_FFT = (np.fft.ifft(zhats,norm="forward"))

    ### Generate associated spectrum to S1D ################################
    Z1D_from_S1D_complex = np.fft.fftshift(np.fft.fft(S1D_from_FFT.real,norm="forward")) # Z33
    Z1D_from_S1D = np.sqrt(np.abs(Z1D_from_S1D_complex)**2)

    ### Envelope squared and envelope ###############################################
    # -- 2D env
    A2D = (S2D_r**2+S2D_i**2) # S1 is real part, S2 is imaginary part
    B2D = np.sqrt(A2D) 
    
    # ---- Spectrum of A = spectrum of envelope squared from 2D Surface ---------------
    Spec_env_2D = np.abs(np.fft.fft2(A2D-np.mean(A2D.flatten()),norm='forward'))**2/(dky*dkx)
    Spec_envB_2D = np.abs(np.fft.fft2(B2D,norm='forward'))**2/(dky*dkx)
    
    # integration over y <=> integration over realisations 
    Spec_env_from_2D = np.sum(Spec_env_2D,axis=0)*dky
    
    Spec_envB_from_2D = np.sum(Spec_envB_2D,axis=0)*dky
    
    Spec_env_from_2D_shifted = np.fft.fftshift(Spec_env_from_2D)

    A1D = S1D_from_FFT.real**2+S1D_from_FFT.imag**2
    B1D = np.sqrt(A1D)
    


    # ---- Spectrum of B = spectrum of enveloppe from 1D Surface ---------------
    # warning, this is a double-sided spectrum
    Spec_env_1D = np.abs(np.fft.fft(A1D,norm='forward'))**2/dkx
    Spec_env_1D_shifted = np.fft.fftshift(Spec_env_1D)

    
    ###  Convolution following Rice pp 131-133.
    #conv_1D = np.convolve(Z1D_def_noshift,Z1D_def_noshift,'full')*dkx
    #conv_1D_pm = conv_1D[nx//2:nx//2+nx]
    conv_1D  = myconv(Z1D_def_noshift ,Z1D_def_noshift )*dkx
    
    # This one is the definition we need: convolution with flipped single-sided spectrum
    conv_1Da  = 4*myconv(Z1D_def_noshift ,np.flip(Z1D_def_noshift) )*dkx
    conv_1Db  = 8*myconv(Z1D_from_FFT_noshift ,Z1D_from_FFT_noshift )*dkx
    
    
    conv_1D_shift  = np.fft.fftshift(conv_1D )
    conv_1Da_shift = np.fft.fftshift(conv_1Da )
    conv_1Db_shift = np.fft.fftshift(conv_1Db )
    
    
    ### Check values of Hs from spectrum #####################################
    # -- original 2D spectrum
    sumZ2D_def=4*np.sqrt(sum(Z2D_def.flatten()*dkx*dky))
    # -- reconstructed (DFT) 2D spectrum
    sumZ2D_from_FFT=4*np.sqrt(sum(Z2D_from_FFT.flatten()*dkx*dky))
    # -- 1D spectrum integrated from original 2D spectrum
    sumZ1D_def = 4*np.sqrt(sum(Z1D_def*dkx))
    # -- 1D spectrum integrated from reconstructed (DFT) 2D spectrum
    sumZ1D_from_FFT = 4*np.sqrt(sum(Z1D_from_FFT*dkx))
    # -- 1D spectrum calculated from 1D surface (from FFT)
    sumZ1D_from_S1D = 4*np.sqrt(sum(Z1D_from_S1D**2))

    ### Envelope stds #####################################
    A1Dstd=np.std(A1D.flatten())
    A2Dstd=np.std(A2D.flatten())
    B1Dstd=np.std(B1D.flatten())
    B2Dstd=np.std(B2D.flatten())
    sum_Specenv1D =4*np.sqrt(sum(Spec_env_1D*dkx))
    sum_conv1Da   =4*np.sqrt(sum(conv_1Da_shift*dkx))
    sum_Specenv1Db=4*np.sqrt(sum(Spec_env_from_2D*dkx))
    sum_SpecenvB1Db=4*np.sqrt(sum(Spec_envB_from_2D*dkx))
    sum_conv1Db   =4*np.sqrt(sum(conv_1Db_shift*dkx))
    
    ### Estimates Hs STD for L > Lc 
    #convc=conv_1Db_shift
    #convc[abs(kx) > kc]=0  # selects wavelengths L > Lc 
    #
    # surface elevation variance E = a^2/2, Hs is 4 * sqrt(E)
    # envelope gives a 
    # envelope squared is a^2 = 2E 
    # hence local Hs is 2*sqrt(2*envelope squared) or 2*sqrt(2)*envelope
    #
    # Now, from the PSD of the envelope the variance of the envelope over a given range 
    # of wavenumber is var_env = sum(convc*dkx), giving a std of the envelope sqrt(sum(convc*dkx))

    std_env_squared=np.sqrt(sum(conv_1Db_shift[abs(kx) < kc])*dkx)
    std_env_approx=std_env_squared/(np.pi*sumZ2D_def/4)
    std_Hs=2*np.sqrt(2)*std_env_approx
    
    
    k1 = np.argmin(np.abs(kx-(-0.03)))
    k2 = np.argmin(np.abs(kx-0.03))
    if isplot:
        ##### spectrum  ######
        fig,axs=plt.subplots(1,1,figsize=(16,6))
        plt.plot(Z1D_def_noshift ,label='Z1D_def_noshift')
        plt.plot(Z1D_from_FFT_noshift ,label='Z1D_from_FFT_noshift')
        plt.legend()
        _=axs.set_title('spectrum no shift')
        
        ##### convolution  ######
        fig,axs=plt.subplots(1,1,figsize=(16,6))
        plt.plot(conv_1D,label='convolution')
        plt.plot(conv_1Da,label='convolution_a')
        plt.plot(conv_1Db,label='convolution_b')
        plt.legend()
        _=axs.set_title('my convolution')
        
        ##### spectrum of enveloppe from 2D and  1D log scale  ######
        fig,axs=plt.subplots(1,1,figsize=(16,6))
        plt.plot(kx,Spec_env_from_2D_shifted,label='ky integration of 2D PSD of 2D envelope')
        plt.plot(kx,Spec_env_1D_shifted,'--',label='PSD of 1D envelope')
        plt.plot(kx,conv_1D_shift,'--',label='convolution')
        plt.plot(kx,conv_1Da_shift,'--',label='convolution a')
        plt.plot(kx,conv_1Db_shift,'--',label='convolution b')
        plt.plot(kx,Z1D_def,'--',label='Z1d spectrum')
        _=axs.set_title('spectrum of 1D envelope')
        plt.grid(True)
        _=plt.ylim(10**-2,10**4)
        # plt.xlim((-0.1,0.1))
        plt.yscale('log')
        plt.xscale('log')
        plt.legend()

                ##### spectrum of enveloppe from 2D and  1D  ######
        fig,axs=plt.subplots(1,1,figsize=(16,6))
        plt.plot(kx,Spec_env_from_2D_shifted,label='ky integration of 2D PSD of 2D envelope')
        plt.plot(kx,Spec_env_1D_shifted,'--',label='PSD of 1D envelope')
        plt.plot(kx,conv_1D_shift,'--',label='convolution')
        plt.plot(kx,conv_1Da_shift,'--',label='convolution a')
        plt.plot(kx,conv_1Db_shift,'--',label='convolution b')
        plt.plot(kx,Z1D_def,'--',label='Z1d spectrum')
        _=axs.set_title('spectrum of 1D envelope')
        plt.grid(True)
        _=plt.ylim(10**-2,10**4)
        # plt.xlim((-0.1,0.1))
        plt.yscale('log')
        plt.legend()


    
    if isprints:
        ##### Prints #####################################################################
        print('---- Hs estimates : ----------------------------------')
        print('4*sqrt(Z2D_ref*dkx*dky) = ',sumZ2D_def)
        print('4*sqrt(Z1D_ref*dkx) = ',sumZ1D_def)
        print('Hs from std (surface 1D from Z1D_def) = ', stdS1D_ref)
        print('')
        print('---- Envelope squared std : ----------------------------------')
        print('4*std(A2D),4*std(A1D) = ',4*A2Dstd,4*A1Dstd)
        print('---- Envelope std (pi*Hs/4 times smaller): -------------------')
        print('4*std(B2D),4*std(B1D) = ',4*B2Dstd,4*B1Dstd,'4*A2Dstd/(pi*Hs/4)',4*A2Dstd/(np.pi*sumZ2D_def/4))
        print('4*sqrt(Specenv1D*dkx) = ',sum_Specenv1D)
        print('4*sqrt(Specenv1Db*dkx) = ',sum_Specenv1Db)
        print('env not squared = ',sum_SpecenvB1Db, sum_SpecenvB1Db/np.sqrt(sum_Specenv1D))
        print('4*sqrt(conv_1Da_pm) = ',sum_conv1Da, 'this is Hs**2 / 2, for L > Lc : ',4*std_env_squared)
        print('4*sqrt(conv_1Db_pm) = ',sum_conv1Db, 'contains harmonics')
        print('--------------------------------------------------------------')
        print('From the wave directional spectrum, we predict std(Hs) for L > 3km: ')
        print(std_Hs)
        
        

In [ ]:
freq_satsampl=40 # freq for waveforms
v_sat=7000 # satellite velocity in m/s
alti_sat=519000 # altitude of satellite CFOSAT
# WARNING: the following 4 parameters should probably be adjusted as a function of the wave height ... 
radi = 4000     # radius used to compute waveform
radi1 = 900     # inner radius for Hs average
radi2 = 1200    # outer radius for Hs average
range_shift = 10 # shift of the waveforms from the "epoch" in meters

isplot_steps=0
# --- edges for range windows ------------
dr = 0.4
edges_max = 20

nHs=251
Hs_max = 25

wfm_ref, Hsm_ref, edges_ref = generate_wvform_database(dr,nHs,edges_max=edges_max,Hs_max=Hs_max,offset=range_shift)

nxa=np.floor(radi/dx).astype(int) # size of radius of footprint in pixel
di=np.floor((v_sat/freq_satsampl)/dx).astype(int) # distance between footprint centers, in pixels (v_sat/freq_satsampl = dsitance in m)
nsamp=np.floor((nx-2*nxa)/di).astype(int) # Nb of samples

Hs_std,Hs_stdbis,Hs_std2,Hs_retrack,Xalt,waveforms,surf1,footprint1 = fly_over_track_v0(X,Y,S2D_r,nsamp,nxa,di,wfm_ref,Hsm_ref,edges_ref,radi,radi1,radi2,alti_sat,range_shift)


In [ ]:
plt.imshow(np.flipud(surf1),vmin=-4, vmax=4)

fig,ax=plt.subplots(figsize=(12,4))
plt.plot(Xalt,Hs_std,color='k',label='python')


In [ ]:
surf2 = np.nan*np.ones(surf1.shape)
surf2[footprint1>0]=surf1[footprint1>0]

Hs_std1 = 4*np.std(surf1)/np.sqrt(np.mean(footprint1))
Hs_std12 = 4*np.nanstd(surf2)#/np.sqrt(np.mean(footprint1))
print('Hs std1 = ',Hs_std1,' - Hs std2 = ',Hs_std12)

In [ ]:

fig,ax=plt.subplots(figsize=(12,4))
plt.plot(edges_ref[0:-1]+dr/2,waveforms[0,:],color='k',label='python')


In [ ]:
fig,ax=plt.subplots(figsize=(12,6))
line1=plt.plot(Xalt*0.001, Hs_retrack, color='k',label='retrack: min')
line2=plt.plot(Xalt*0.001, Hs_std, color='r',label='Hs avg on disk')
line3=plt.plot(Xalt*0.001, Hs_std2, color='g',label='Hs avg on annulus')
plt.grid
plt.xlabel('x (km)')
plt.ylabel('Hs (m)')
plt.title('Different estimates of wave height')
leg = plt.legend(loc='upper right')
plt.show()